# VGG16

In [1]:
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.init as init
import torchvision.datasets as dset
import torchvision.transforms as transforms
from torch.utils.data import DataLoader
from torch.autograd import Variable
import matplotlib.pyplot as plt
import matplotlib.gridspec as gridspec
import torchvision.models as models
import torchvision 
% matplotlib inline

In [2]:
transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=128,
                                          shuffle=False, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=128,
                                         shuffle=False, num_workers=2)

classes = ('airplane', 'automobile', 'bird', 'cat', 'deer',
                   'dog', 'frog', 'horse', 'ship', 'truck')


Files already downloaded and verified
Files already downloaded and verified


In [3]:
class vgg16_fine(nn.Module):
            def __init__(self):
                super(vgg16_fine, self).__init__()
                self.features = nn.Sequential(
                    nn.Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(64),
                    nn.Dropout(0.3),
                    nn.Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(64),
                    nn.MaxPool2d (2,2),
                    nn.Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(128),
                    nn.Dropout(0.4),
                    nn.Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(128),
                    nn.MaxPool2d (2,2),
                    nn.Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(256),
                    nn.Dropout(0.4),
                    nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(256),
                    nn.Dropout(0.4),
                    nn.Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(256),
                    nn.MaxPool2d (2,2),
                    nn.Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(512),
                    nn.Dropout(0.4),
                    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(512),
                    nn.Dropout(0.4),
                    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(512),
                    nn.MaxPool2d (2,2),
                    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(512),
                    nn.Dropout(0.4),
                    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(512),
                    nn.Dropout(0.4),
                    nn.Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1)),
                    nn.ReLU (inplace=True),                    
                    nn.BatchNorm2d(512),
                    nn.MaxPool2d (2,2),
                    nn.Dropout(0.5)) 
                self.classifier = nn.Sequential(
                 nn.Linear(512, 512),
                 nn.ReLU(inplace=True),
                 nn.BatchNorm2d(512),
                 nn.Dropout(0.5),
                 nn.Linear(512, 10),
                 nn.Softmax()   
            )
            def forward(self, x):
                f = self.features(x)
                f=f.view(-1,512*1*1)
                y=self.classifier(f)
                return y

vgg16 = vgg16_fine()

In [4]:
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(vgg16.parameters(), lr=0.001)


In [5]:
for epoch in range(1):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(trainloader, 0):
        # get the inputs
        inputs, labels = data

        # wrap them in Variable
        inputs, labels = Variable(inputs), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = vgg16(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        # print statistics
        running_loss += loss.data[0]
        if i % 50 == 0:    # print every 50 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 50))
            running_loss = 0.0
            
        if i%200==0:
            torch.save(vgg16,'vgg16_'+str(i)+'.pkl')
            torch.save(vgg16,'vgg16.pkl')   

print('Finished Training')

[1,     1] loss: 0.046
[1,    51] loss: 2.250
[1,   101] loss: 2.207
[1,   151] loss: 2.182
[1,   201] loss: 2.144
[1,   251] loss: 2.115
[1,   301] loss: 2.092
[1,   351] loss: 2.056
Finished Training


In [6]:
correct = 0
total = 0
for data in testloader:
    images, labels = data
    outputs = vgg16(Variable(images))
    _, predicted = torch.max(outputs.data, 1)
    total += labels.size(0)
    correct += (predicted == labels).sum()

print('Accuracy of the network on the 10000 test images: %d %%' % (
    100 * correct / total))

Accuracy of the network on the 10000 test images: 41 %
